In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Reshape, Concatenate, Activation
from tensorflow.keras.utils import plot_model
from transformers import TFAutoModel
from tensorflow.keras import backend as K
from focal_loss import sparse_categorical_focal_loss
from transformers import AutoModel
from tensorflow.keras.layers import concatenate
from keras_contrib.layers import CRF
# import tensorflow_hub as hub
# import tensorflow_text as text
import pythainlp
import spacy_thai
from nltk.tokenize import RegexpTokenizer
import re
import string

In [3]:
train = pd.read_csv('data/ohm_data/train.csv')
test = pd.read_csv('data/ohm_data/test.csv')

columns = ['E1', 'E2', 'before_E1', 'before_E2', 'after_E1', 'after_E2', 'between_E1_E2']

for col in columns:
    train[col] = train[col].str.strip('[]').str.split(', ').apply(lambda x: [int(i) for i in x])

for col in columns:
    test[col] = test[col].str.strip('[]').str.split(', ').apply(lambda x: [int(i) for i in x])
train.head()

,Unnamed: 0,text,E1,E2,E1_entity,E2_entity,before_E1,before_E2,after_E1,after_E2,between_E1_E2,label
0,469,วันที่ 16 ม.ค.2565 ตำรวจสภ.เฝ้าไร่ จ.หนองคายรั...,"[4862, 3510, 1, 6215, 11081, 12440, 12440, 124...","[5297, 4840, 9906, 12440, 12440, 12440, 12440,...",6,2,"[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[1, 9711, 1, 520, 1, 5721, 5001, 2095, 5975, 1...","[1, 3704, 10744, 12341, 1, 2988, 4887, 1901, 1...","[5297, 4840, 9906, 1, 3704, 10744, 12341, 1, 2...",7
1,10928,เมื่อเวลาประมาณ 09.30 น.วันที่ 11 ก.ค. 64 เจ้า...,"[4862, 9811, 1, 11639, 8227, 7468, 12440, 1244...","[5297, 10957, 1, 1104, 103, 425, 12440, 12440,...",6,2,"[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[1, 9711, 1, 953, 1, 5721, 1, 9545, 5297, 1095...","[1, 4089, 8947, 7896, 2742, 1, 3928, 9374, 700...","[5297, 10957, 1, 1104, 103, 425, 1, 4089, 8947...",2
2,11885,เวลา 20.00 น. เมื่อวันที่ 19 ม.ค. 65 พ.ต.อ.พงศ...,"[11911, 1, 7059, 8121, 12031, 12440, 12440, 12...","[9424, 10957, 1, 460, 12440, 12440, 12440, 124...",4,2,"[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[1, 181, 1, 10115, 1, 11653, 7161, 2835, 1, 42...","[1, 3188, 1, 425, 1, 7100, 9519, 1, 9177, 1, 1...","[11911, 1, 7059, 8121, 12031, 1, 181, 1, 10115...",6
3,9897,จากกรณีเหตุคนร้ายก่อเหตุยิงนายอนันตชัย จริตรัม...,"[4862, 2978, 8192, 1, 9806, 4828, 5150, 12440,...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",6,2,"[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[1, 9711, 1, 676, 1, 5721, 1, 3220, 6617, 1, 1...","[1, 9461, 8651, 1, 2806, 3135, 1, 8207, 3135, ...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",2
4,5460,มื่อเวลา 18.00 น. วันที่ 18 ม.ค. 65 ผู้สื่อข่า...,"[4862, 5120, 12440, 12440, 12440, 12440, 12440...","[4862, 7890, 7409, 8136, 12440, 12440, 12440, ...",6,6,"[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[1, 14, 4859, 8438, 48, 1, 9711, 1, 341, 1, 57...","[1, 9711, 1, 341, 1, 5721, 1, 2920, 1747, 8583...","[4862, 7890, 7409, 8136, 1, 9711, 1, 341, 1, 5...",3


In [4]:
train.head()


,Unnamed: 0,text,E1,E2,E1_entity,E2_entity,before_E1,before_E2,after_E1,after_E2,between_E1_E2,label
0,469,วันที่ 16 ม.ค.2565 ตำรวจสภ.เฝ้าไร่ จ.หนองคายรั...,"[4862, 3510, 1, 6215, 11081, 12440, 12440, 124...","[5297, 4840, 9906, 12440, 12440, 12440, 12440,...",6,2,"[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[1, 9711, 1, 520, 1, 5721, 5001, 2095, 5975, 1...","[1, 3704, 10744, 12341, 1, 2988, 4887, 1901, 1...","[5297, 4840, 9906, 1, 3704, 10744, 12341, 1, 2...",7
1,10928,เมื่อเวลาประมาณ 09.30 น.วันที่ 11 ก.ค. 64 เจ้า...,"[4862, 9811, 1, 11639, 8227, 7468, 12440, 1244...","[5297, 10957, 1, 1104, 103, 425, 12440, 12440,...",6,2,"[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[1, 9711, 1, 953, 1, 5721, 1, 9545, 5297, 1095...","[1, 4089, 8947, 7896, 2742, 1, 3928, 9374, 700...","[5297, 10957, 1, 1104, 103, 425, 1, 4089, 8947...",2
2,11885,เวลา 20.00 น. เมื่อวันที่ 19 ม.ค. 65 พ.ต.อ.พงศ...,"[11911, 1, 7059, 8121, 12031, 12440, 12440, 12...","[9424, 10957, 1, 460, 12440, 12440, 12440, 124...",4,2,"[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[1, 181, 1, 10115, 1, 11653, 7161, 2835, 1, 42...","[1, 3188, 1, 425, 1, 7100, 9519, 1, 9177, 1, 1...","[11911, 1, 7059, 8121, 12031, 1, 181, 1, 10115...",6
3,9897,จากกรณีเหตุคนร้ายก่อเหตุยิงนายอนันตชัย จริตรัม...,"[4862, 2978, 8192, 1, 9806, 4828, 5150, 12440,...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",6,2,"[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[1, 9711, 1, 676, 1, 5721, 1, 3220, 6617, 1, 1...","[1, 9461, 8651, 1, 2806, 3135, 1, 8207, 3135, ...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",2
4,5460,มื่อเวลา 18.00 น. วันที่ 18 ม.ค. 65 ผู้สื่อข่า...,"[4862, 5120, 12440, 12440, 12440, 12440, 12440...","[4862, 7890, 7409, 8136, 12440, 12440, 12440, ...",6,6,"[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[1, 14, 4859, 8438, 48, 1, 9711, 1, 341, 1, 57...","[1, 9711, 1, 341, 1, 5721, 1, 2920, 1747, 8583...","[4862, 7890, 7409, 8136, 1, 9711, 1, 341, 1, 5...",3


In [5]:
# label = dict([(v, k) for k,v in enumerate(list(train['label'].unique()))])
# train['label'] = train['label'].apply(lambda x: label[x])

# test['label'] = test['label'].apply(lambda x: label[x])

In [6]:
label

NameError: name 'label' is not defined

In [ ]:
train['E2_entity'].unique()

In [18]:
input_dim_long = 12440 + 1
input_dim_ent = 10

input_len_long = len(train['before_E1'][0])
input_len_ent = 1

n_tags = 12
output_dim = 8

model_E1 = Input(shape = (input_len_long,))
emb_E1 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_E1)
output_E1 = Reshape(target_shape=(output_dim, input_len_long))(emb_E1)

model_E2 = Input(shape = (input_len_long,))
emb_E2 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_E2)
output_E2 = Reshape(target_shape=(output_dim, input_len_long))(emb_E2)

model_ent_E1 = Input(shape = (input_len_ent,))
emb_ent_E1 = Embedding(input_dim=input_dim_ent, output_dim=output_dim)(model_ent_E1)
output_ent_E1 = Reshape(target_shape=(output_dim, input_len_ent))(emb_ent_E1)

model_ent_E2 = Input(shape = (input_len_ent,))
emb_ent_E2 = Embedding(input_dim=input_dim_ent, output_dim=output_dim)(model_ent_E2)
output_ent_E2 = Reshape(target_shape=(output_dim, input_len_ent))(emb_ent_E2)

model_b4_E1 = Input(shape = (input_len_long,))
emb_b4_E1 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_b4_E1)
output_b4_E1 = Reshape(target_shape=(output_dim, input_len_long))(emb_b4_E1)

model_b4_E2 = Input(shape = (input_len_long,))
emb_b4_E2 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_b4_E2)
output_b4_E2 = Reshape(target_shape=(output_dim, input_len_long))(emb_b4_E2)

model_at_E1 = Input(shape = (input_len_long,))
emb_at_E1 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_at_E1)
output_at_E1 = Reshape(target_shape=(output_dim, input_len_long))(emb_at_E1)

model_at_E2 = Input(shape = (input_len_long,))
emb_at_E2 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_at_E2)
output_at_E2 = Reshape(target_shape=(output_dim, input_len_long))(emb_at_E2)

model_btw_E1_E2 = Input(shape = (input_len_long,))
emb_btw_E1_E2 = Embedding(input_dim=input_dim_long, output_dim=output_dim)(model_btw_E1_E2)
output_btw_E1_E2 = Reshape(target_shape=(output_dim, input_len_long))(emb_btw_E1_E2)

input_model = [model_E1, model_E2,
               model_ent_E1, model_ent_E2,
               model_b4_E1, model_b4_E2,
               model_at_E1, model_at_E2,
              
               model_btw_E1_E2]

output_embeddings = [output_E1, output_E2,
               output_ent_E1, output_ent_E2,
               output_b4_E1, output_b4_E2,
               output_at_E1, output_at_E2,
                    
               output_btw_E1_E2]

output_model = Concatenate()(output_embeddings)
output_model = Bidirectional(LSTM(units=output_dim, return_sequences=False, dropout=0.5, recurrent_dropout=0.5))(output_model)
output_model = Dense(n_tags, activation = 'softmax')(output_model)

model = Model(inputs = input_model, outputs = output_model)
    
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 630)]        0           []                               
                                                                                                  
 input_38 (InputLayer)          [(None, 630)]        0           []                               
                                                                                                  
 input_39 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_40 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [40]:
train

,Unnamed: 0,text,E1,E2,E1_entity,E2_entity,before_E1,before_E2,after_E1,after_E2,between_E1_E2,label
0,469,วันที่ 16 ม.ค.2565 ตำรวจสภ.เฝ้าไร่ จ.หนองคายรั...,"[4869, 3518, 1, 6224, 11091, 12453, 12453, 124...","[340, 1, 6633, 571, 12453, 12453, 12453, 12453...",6,1,"[7994, 1, 340, 1, 6633, 571, 1, 3935, 8427, 10...","[7994, 1, 12453, 12453, 12453, 12453, 12453, 1...","[1, 9721, 1, 519, 1, 5730, 5008, 2096, 5984, 1...","[1, 3935, 8427, 10754, 12354, 1, 2809, 9075, 7...","[340, 1, 6633, 571, 1, 3935, 8427, 10754, 1235...",1
1,10928,วันที่ 20 ม.ค.65 ที่กองอำนวยการรักษาความมั่นคง...,"[4869, 7257, 3372, 7795, 1, 10226, 10983, 1245...","[7603, 1, 8956, 1, 5226, 3457, 7963, 12453, 12...",6,6,"[7994, 1, 436, 1, 6633, 909, 1, 4494, 1675, 73...","[7994, 1, 436, 1, 6633, 909, 1, 4494, 1675, 73...","[1, 9721, 1, 735, 1, 5730, 1, 6840, 9170, 1, 5...","[1, 4169, 1500, 7050, 4288, 8621, 3695, 3436, ...","[7603, 1, 8956, 1, 5226, 3457, 7963, 1, 4169, ...",3
2,11885,ตำรวจเข้าคุมตัว นายสุริยา เริ่มสมุทร หรือบังฟิ...,"[5133, 6519, 12453, 12453, 12453, 12453, 12453...","[2809, 1491, 12453, 12453, 12453, 12453, 12453...",6,2,"[3935, 10079, 2679, 1, 4869, 8958, 1, 10914, 8...","[3935, 10079, 2679, 1, 4869, 8958, 1, 10914, 8...","[1, 9721, 1, 640, 1, 5730, 1, 5931, 12138, 230...","[1, 9262, 4210, 11655, 12330, 12400, 11664, 39...","[5133, 6519, 1, 9721, 1, 640, 1, 5730, 1, 5931...",8
3,9897,เวลา 21.00 น.วันที่ 28 ต.ค. 64 พันเอก สุดเขตต์...,"[4869, 8249, 1405, 1, 8238, 7479, 7518, 8124, ...","[8427, 10868, 10317, 12453, 12453, 12453, 1245...",6,5,"[11053, 1, 460, 1, 4699, 7994, 1, 596, 1, 3711...","[11053, 1, 460, 1, 4699, 7994, 1, 596, 1, 3711...","[1, 9721, 1, 629, 1, 5730, 1, 11664, 1, 4869, ...","[1, 10805, 4412, 2069, 1771, 1, 5896, 7563, 20...","[4869, 8249, 1405, 1, 8238, 7479, 7518, 8124, ...",8
4,5460,เมื่อวันที่ 17 ธ.ค. พ.ต.ท.ปิยพัชร์ ปลาทอง สว.(...,"[4869, 8899, 1914, 1, 10992, 5471, 1960, 4222,...","[4869, 5398, 1, 11107, 8459, 12453, 12453, 124...",6,6,"[10870, 7994, 1, 361, 1, 4625, 1, 6063, 5723, ...","[10870, 7994, 1, 361, 1, 4625, 1, 6063, 5723, ...","[1, 9721, 1, 1022, 1, 5730, 1, 5906, 11285, 1,...","[1, 9721, 1, 826, 1, 5730, 1, 4169, 7050, 1008...","[4869, 8899, 1914, 1, 10992, 5471, 1960, 4222,...",3
...,...,...,...,...,...,...,...,...,...,...,...,...
10407,11964,ร.ต.ท.ชัลวาลย์ ไต๋เจริญ รอง สวป.สน.บางขุนเทียน...,"[5567, 12453, 12453, 12453, 12453, 12453, 1245...","[10040, 6602, 10220, 12453, 12453, 12453, 1245...",4,2,"[7149, 3187, 8043, 7143, 1, 12263, 10220, 1, 7...","[7149, 3187, 8043, 7143, 1, 12263, 10220, 1, 7...","[5741, 1, 4494, 7050, 12289, 11668, 1, 800, 1,...","[1, 1387, 1, 4335, 1, 1771, 3579, 1, 8887, 105...","[5566, 1500, 1, 4494, 7050, 12289, 11668, 1, 8...",6
10408,5191,คดีที่มีคนร้ายใช้อาวุธปืนกระหน่ำยิงชายอายุ 45 ...,"[4869, 4639, 10382, 1, 8835, 7337, 4650, 12453...","[7176, 12453, 12453, 12453, 12453, 12453, 1245...",6,8,"[2300, 4494, 6840, 2327, 12119, 9734, 5741, 15...","[2300, 4494, 6840, 2327, 12119, 9734, 5741, 15...","[1, 2875, 12119, 9734, 5741, 2990, 5135, 9065,...","[7382, 12178, 6339, 9232, 1, 6230, 1, 2981, 86...","[4869, 4639, 10382, 1, 8835, 7337, 4650, 1, 28...",9
10409,5390,เมื่อเวลา 12.30 น. วันที่ 31 ธันวาคม 2564 ร.ต....,"[7190, 11899, 1, 6501, 12453, 12453, 12453, 12...","[728, 1, 1351, 726, 1, 1550, 12453, 12453, 124...",8,3,"[10870, 11053, 1, 261, 1, 4699, 1, 7994, 1, 64...","[10870, 11053, 1, 261, 1, 4699, 1, 7994, 1, 64...","[1, 8857, 3609, 1, 9168, 10966, 1, 728, 1, 135...","[1, 9553, 12138, 8431, 3106, 1716, 3810, 5077,...","[7190, 11899, 1, 6501, 1, 8857, 3609, 1, 9168,...",11
10410,860,ทางเจ้าหน้าที่ได้นำรถออกไล่ติดตามพร้อมทั้งประส...,"[3709, 7023, 9910, 12453, 12453, 12453, 12453,...","[7190, 12453, 12453, 12453, 12453, 12453, 1245...",2,8,"[4345, 10262, 12228, 4894, 7172, 9600, 12364, ...","[4345, 10262, 12228, 4894, 7172, 9600, 12364, ...","[1, 2821, 8716, 8310, 2913, 7172, 2575, 3579, ...","[4494, 11216, 9795, 1, 619, 1, 2575, 4043, 0, ...","[3709, 7023, 9910, 1, 2821, 8716, 8310,

In [19]:
columns = ['E1', 'E2', 'E1_entity', 'E2_entity', 'before_E1', 'before_E2', 'after_E1', 'after_E2', 'between_E1_E2']

X_train = []

for col in columns:
    col_list = []
    for i in train[col]:
        col_list.append(i)
    X_train.append(np.array(col_list))
    
model.fit(X_train, train['label'], batch_size = 128, epochs = 20)

Epoch 1/20
82/82 [==============================] - 194s 2s/step - loss: 2.1112 - accuracy: 0.3031
Epoch 2/20
82/82 [==============================] - 157s 2s/step - loss: 1.6001 - accuracy: 0.5208
Epoch 3/20
82/82 [==============================] - 162s 2s/step - loss: 1.0544 - accuracy: 0.7197
Epoch 4/20
82/82 [==============================] - 164s 2s/step - loss: 0.6783 - accuracy: 0.8366
Epoch 5/20
82/82 [==============================] - 161s 2s/step - loss: 0.4693 - accuracy: 0.9005
Epoch 6/20
82/82 [==============================] - 153s 2s/step - loss: 0.3478 - accuracy: 0.9321
Epoch 7/20
82/82 [==============================] - 151s 2s/step - loss: 0.2601 - accuracy: 0.9554
Epoch 8/20
82/82 [==============================] - 152s 2s/step - loss: 0.2012 - accuracy: 0.9670
Epoch 9/20
82/82 [==============================] - 153s 2s/step - loss: 0.1612 - accuracy: 0.9731
Epoch 10/20
82/82 [==============================] - 157s 2s/step - loss: 0.1331 - accuracy: 0.9789
Epoch 11/

In [20]:
X_test = []

for col in columns:
    col_list = []
    for i in test[col]:
        col_list.append(i)
    X_test.append(np.array(col_list))
y_pred = np.argmax(model.predict(X_test), axis = 1)


In [21]:
train[train['E1'] == ]

,Unnamed: 0,text,E1,E2,E1_entity,E2_entity,before_E1,before_E2,after_E1,after_E2,between_E1_E2,label
0,469,วันที่ 16 ม.ค.2565 ตำรวจสภ.เฝ้าไร่ จ.หนองคายรั...,"[4862, 3510, 1, 6215, 11081, 12440, 12440, 124...","[5297, 4840, 9906, 12440, 12440, 12440, 12440,...",6,2,"[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[7984, 1, 341, 1, 6623, 573, 1, 3929, 8417, 10...","[1, 9711, 1, 520, 1, 5721, 5001, 2095, 5975, 1...","[1, 3704, 10744, 12341, 1, 2988, 4887, 1901, 1...","[5297, 4840, 9906, 1, 3704, 10744, 12341, 1, 2...",7
1,10928,เมื่อเวลาประมาณ 09.30 น.วันที่ 11 ก.ค. 64 เจ้า...,"[4862, 9811, 1, 11639, 8227, 7468, 12440, 1244...","[5297, 10957, 1, 1104, 103, 425, 12440, 12440,...",6,2,"[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[10859, 11043, 5470, 1, 179, 1, 4691, 7984, 1,...","[1, 9711, 1, 953, 1, 5721, 1, 9545, 5297, 1095...","[1, 4089, 8947, 7896, 2742, 1, 3928, 9374, 700...","[5297, 10957, 1, 1104, 103, 425, 1, 4089, 8947...",2
2,11885,เวลา 20.00 น. เมื่อวันที่ 19 ม.ค. 65 พ.ต.อ.พงศ...,"[11911, 1, 7059, 8121, 12031, 12440, 12440, 12...","[9424, 10957, 1, 460, 12440, 12440, 12440, 124...",4,2,"[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[11043, 1, 441, 1, 4691, 1, 10859, 7984, 1, 40...","[1, 181, 1, 10115, 1, 11653, 7161, 2835, 1, 42...","[1, 3188, 1, 425, 1, 7100, 9519, 1, 9177, 1, 1...","[11911, 1, 7059, 8121, 12031, 1, 181, 1, 10115...",6
3,9897,จากกรณีเหตุคนร้ายก่อเหตุยิงนายอนันตชัย จริตรัม...,"[4862, 2978, 8192, 1, 9806, 4828, 5150, 12440,...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",6,2,"[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[2920, 1413, 11164, 2325, 2000, 7040, 4862, 94...","[1, 9711, 1, 676, 1, 5721, 1, 3220, 6617, 1, 1...","[1, 9461, 8651, 1, 2806, 3135, 1, 8207, 3135, ...","[7731, 3741, 6164, 7532, 10042, 3249, 2836, 72...",2
4,5460,มื่อเวลา 18.00 น. วันที่ 18 ม.ค. 65 ผู้สื่อข่า...,"[4862, 5120, 12440, 12440, 12440, 12440, 12440...","[4862, 7890, 7409, 8136, 12440, 12440, 12440, ...",6,6,"[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[6903, 11043, 1, 380, 1, 4691, 1, 7984, 1, 378...","[1, 14, 4859, 8438, 48, 1, 9711, 1, 341, 1, 57...","[1, 9711, 1, 341, 1, 5721, 1, 2920, 1747, 8583...","[4862, 7890, 7409, 8136, 1, 9711, 1, 341, 1, 5...",3
...,...,...,...,...,...,...,...,...,...,...,...,...
10407,11964,เจ้าหน้าที่กองพิสูจน์หลักฐาน แพทย์ และตำรวจ สน...,"[3025, 8824, 9506, 1, 2430, 6122, 12440, 12440...","[5732, 12440, 12440, 12440, 12440, 12440, 1244...",6,9,"[10252, 1666, 6401, 9248, 1, 11567, 1, 11653, ...","[10252, 1666, 6401, 9248, 1, 11567, 1, 11653, ...","[1, 32, 9724, 5732, 4488, 6893, 2254, 2079, 48...","[4488, 6893, 2254, 2079, 48, 1, 4845, 8077, 27...","[3025, 8824, 9506, 1, 2430, 6122, 1, 32, 9724,...",10
10408,5191,วันนี้ (8 ม.ค. 65) เวลา 14.00 น. ที่สำนักงานตำ...,"[1000, 1, 6623, 1, 912, 12440, 12440, 12440, 1...","[621, 1, 6623, 912, 12440, 12440, 12440, 12440...",1,1,"[7986, 1, 14, 12440, 12440, 12440, 12440, 1244...","[7986, 1, 14, 1000, 1, 6623, 1, 912, 48, 1, 11...","[48, 1, 11043, 1, 297, 1, 4691, 1, 4488, 8774,...","[1, 10755, 10672, 4488, 5415, 10383, 1587, 1, ...","[1000, 1, 6623, 1, 912, 48, 1, 11043, 1, 297, ...",8
10409,5390,ผู้สื่อข่าวรายงานว่า เมื่อเวลาประมาณ 02.30 น ....,"[8417, 5767, 6280, 12440, 12440, 12440, 12440,...","[5297, 10957, 1, 536, 12440, 12440, 12440, 124...",5,2,"[5960, 7444, 8142, 1, 10859, 11043, 5470, 1, 1...","[5960, 7444, 8142, 1, 10859, 11043, 5470, 1, 1...","[1, 6055, 4624, 8812, 1, 2808, 9297, 1, 5795, ...","[1, 9177, 4488, 1, 425, 1, 3704, 7730, 2233, 1...","[8417, 5767, 6280, 1, 6055, 4624, 8812, 1, 280...",8
10410,860,วันที่ 24 ต.ค.2562 นายรณณรงค์ แก้วเพ็ชร์ ประธา...,"[4862, 2875, 1575, 1, 11639, 11074, 6621, 1244...","[5737, 2051, 1, 59, 660, 12440, 12440, 12440, ...",6,9,"[7984, 1, 520, 1, 3705, 568, 1, 4862, 7153, 34...","[7984, 1, 520, 1, 3705, 568, 1, 4862, 7153, 34...","[1, 9711, 1, 710, 1, 5721, 1, 3395, 10664, 870...","[1, 7040, 12158, 11601, 10079, 5100, 9338, 263...","[4862, 2875, 1575, 1, 11639, 11074, 662

In [22]:
from sklearn.metrics import classification_report


print(classification_report(test['label'], 
                            y_pred,
                           )
     )
    

              precision    recall  f1-score   support

           0       0.94      0.98      0.96        86
           1       0.88      0.95      0.91       103
           2       0.93      0.95      0.94       489
           3       0.92      0.96      0.94       141
           4       0.95      0.93      0.94        42
           5       0.87      0.93      0.90       132
           6       0.99      0.99      0.99       330
           7       0.92      0.92      0.92       411
           8       0.94      0.86      0.90       581
           9       0.91      0.89      0.90        90
          10       0.89      0.94      0.91       146
          11       0.95      1.00      0.97        53

    accuracy                           0.93      2604
   macro avg       0.92      0.94      0.93      2604
weighted avg       0.93      0.93      0.93      2604



In [12]:
model.save('model/REL/REL_model_v2_27_1_2022.h5')

In [11]:
test_ = test.copy()
test_['pred'] = y_pred

reverse_label = dict([(v, k) for k,v in label.items()])
test_['pred'] = test_['pred'].apply(lambda x: reverse_label[x])
test_['label'] = test_['label'].apply(lambda x: reverse_label[x])

test_

NameError: name 'label' is not defined